In [1]:
import requests
import json
from rich import print_json

In [8]:
def lanceRequete (collection, req):
    reponses = requests.get("https://api.archives-ouvertes.fr/search/"+collection+"/?q="+req)
    #print_json(json.dumps(json.loads(reponses.text), indent=3))
    print_json(json.dumps(json.loads(reponses.text), indent=3))

### les champs des requêtes :
fq définit les filtres (par exemple une plage)
facet définit les facettes qui permettent de ventiler les réponses en groupes


# Liste des labos INS2I

In [2]:
listeLabosINS2I=("CMP", "CRAN", "CREATIS", "CRIL","CRISTAL","DI ENS","ETIS","GIPSA-Lab","GREYC","G-SCOP","HEUDIASYC","I3S","ICUBE","IDRIS","IRIF","IRISA","IRIT","L2S","LAAS","LABRI","LAB-STICC","LAMIH","LAMSADE","LIG","LIGM","LIMOS","LISN","LIP","LIP6","LIPN","LIRIS","LIRMM","LIS","LIX","LORIA","LS2N","LSV","STMS","TIMA","TIMC IMAG","VERIMAG")

In [113]:
recherche = 'authFullName_t:Patrice+Bellot&sort=publicationDate_s desc&indent=true&rows=10&fl=label_s,keyword_s&indent=true&facet=true&facet.field=keyword_s&wt=json'

In [114]:
reponses = requests.get("https://api.archives-ouvertes.fr/search/?q="+recherche)

In [136]:
j = json.loads(reponses.text)
print("Nombre documents trouvés : ",j['response']['numFound'])

Nombre documents trouvés :  145


In [115]:
print(reponses.text)

{
  "response":{"numFound":145,"start":0,"docs":[
      {
        "label_s":"Liana Ermakova, Patrice Bellot, Jaap Kamps, Diana Nurbakova, Irina Ovchinnikova, et al.. Automatic Simplification of Scientific Texts: SimpleText Lab at CLEF-2022. Advances in Information Retrieval, 13186, Springer International Publishing, pp.364-373, 2022, Lecture Notes in Computer Science, &#x27E8;10.1007/978-3-030-99739-7_46&#x27E9;. &#x27E8;hal-03637775&#x27E9;"},
      {
        "label_s":"Haytame Fallah, Patrice Bellot, Emmanuel Bruno, Elisabeth Murisasco. Architectures Transformeurs pour la classification multilabels de textes. BDA 2021 - 37ème Conférence sur la Gestion de Données – Principes, Technologies et Applications, Oct 2021, Paris, France. &#x27E8;hal-03489418&#x27E9;",
        "keyword_s":["Deep learning",
          "Transformers",
          "Language models",
          "Multilabel text classification",
          "BERT",
          "Deep Learning",
          "Apprentissage profond",
          "

-------------------------------------

In [12]:
recherche = "labStructAcronym_s:LIRIS&rows=10&fq=producedDateY_i:[2016 TO 2021]&wt=json"

In [13]:
reponses = requests.get("https://api.archives-ouvertes.fr/search/CNRS/?q="+recherche)

In [14]:
print(json.dumps(json.loads(reponses.text), indent=3))

{
   "response": {
      "numFound": 2181,
      "start": 0,
      "docs": [
         {
            "docid": 1878010,
            "label_s": "Abir Masmoudi, Mahmoud Barhamgi, Noura Faci, Zohra Saoud, Khalid Belhajjame, et al.. An Ontology-Based Approach for Mining Radicalization Indicators from Online Messages. 2018 IEEE 32nd International Conference on Advanced Information Networking and Applications (AINA), May 2018, Krakow, Poland. pp.609-616, &#x27E8;10.1109/AINA.2018.00094&#x27E9;. &#x27E8;hal-01878010&#x27E9;",
            "uri_s": "https://hal.archives-ouvertes.fr/hal-01878010"
         },
         {
            "docid": 3390102,
            "label_s": "Alexandre Millot. Exceptional Model Mining meets Multi-Objective Optimization: Application to Plant Growth Recipes in Controlled Environments. Computer Science [cs]. INSA Lyon, 2021. English. &#x27E8;tel-03390102&#x27E9;",
            "uri_s": "https://hal.archives-ouvertes.fr/tel-03390102"
         },
         {
            "doc

## La liste des auteurs du LIRIS entre 2016 et 2020

In [62]:
lanceRequete("CNRS","labStructAcronym_s:LIRIS*&fq=producedDateY_i:[2016%20TO%202020]&rows=0&&facet=true&facet.field=structHasAuthId_fs&facet.prefix=2003_FacetSep_")

{
  "response": {
    "numFound": 1871,
    "start": 0,
    "docs": []
  },
  "facet_counts": {
    "facet_queries": {},
    "facet_fields": {
      "structHasAuthId_fs": [
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_674908_FacetSep_Djamal Benslimane",
        46,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_307062_FacetSep_Christian Wolf",
        43,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_757194_FacetSep_Samir Aknine",
        37,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_46964_FacetSep_Guillaume Beslon",
        36,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1668775_FacetSep_Eric Tannier",
        32,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_212248_FacetSep_David Coeurjolly",
        32,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1247523_FacetSep_Parisa Ghodous",
        30,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1224140_FacetSep_Noura Faci",
        29,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1200816_FacetSep_Angela Bonifati",
        28,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_693447_FacetSep_Mahmoud Barhamgi",
        28,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1108819_FacetSep_Sonia Ben Mokhtar",
        27,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1011685_FacetSep_Stefan Duffner",
        26,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_184772_FacetSep_Christine Solnon",
        26,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1113630_FacetSep_Mehdi Kaytoue",
        25,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_314872_FacetSep_C\u00e9line Robardet",
        25,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_374940_FacetSep_Youakim Badr",
        24,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_855381_FacetSep_Liming Chen",
        24,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_222208_FacetSep_Laure Tougne",
        23,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_298519_FacetSep_Christophe Garcia",
        23,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_620262_FacetSep_Chirine Ghedira",
        23,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_795840_FacetSep_Sara Bouchenak",
        23,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_1164964_FacetSep_Elise Lavou\u00e9",
        21,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_167613_FacetSep_Sylvie Servigne",
        21,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_195084_FacetSep_La\u00ebtitia Matignon",
        21,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_206969_FacetSep_Hamamache Kheddouci",
        21,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information_JoinSep_439784_FacetSep_Christophe Rigotti",
        21,
        "2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8

In [63]:
lanceRequete("CNRS","labStructAcronym_s:LIRIS*&fq=producedDateY_i:[2016%20TO%202020]&facet=true&facet.pivot=submitType_s,authIdHasPrimaryStructure_fs&rows=0&wt=json")

{
  "response": {
    "numFound": 1871,
    "start": 0,
    "docs": []
  },
  "facet_counts": {
    "facet_queries": {},
    "facet_fields": {},
    "facet_ranges": {},
    "facet_intervals": {},
    "facet_heatmaps": {},
    "facet_pivot": {
      "submitType_s,authIdHasPrimaryStructure_fs": [
        {
          "field": "submitType_s",
          "value": "file",
          "count": 1083,
          "pivot": [
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "46964_FacetSep_Guillaume Beslon_JoinSep_160051_FacetSep_Artificial Evolution and Computational Biology",
              "count": 32
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "46964_FacetSep_Guillaume Beslon_JoinSep_2003_FacetSep_Laboratoire d'InfoRmatique en Image et Syst\u00e8mes d'information",
              "count": 30
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1108819_FacetSep_Sonia Ben Mokhtar_JoinSep_394737_FacetSep_Distribution, Recherche d'Information et Mobilit\u00e9",
              "count": 24
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1113630_FacetSep_Mehdi Kaytoue_JoinSep_413086_FacetSep_Data Mining and Machine Learning",
              "count": 24
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1668775_FacetSep_Eric Tannier_JoinSep_160051_FacetSep_Artificial Evolution and Computational Biology",
              "count": 24
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1011685_FacetSep_Stefan Duffner_JoinSep_403930_FacetSep_Extraction de Caract\u00e9ristiques et Identification",
              "count": 23
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "307062_FacetSep_Christian Wolf_JoinSep_403930_FacetSep_Extraction de Caract\u00e9ristiques et Identification",
              "count": 23
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "184772_FacetSep_Christine Solnon_JoinSep_391965_FacetSep_Geometry Processing and Constrained Optimization",
              "count": 22
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1247523_FacetSep_Parisa Ghodous_JoinSep_393309_FacetSep_Service Oriented Computing",
              "count": 18
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "212248_FacetSep_David Coeurjolly_JoinSep_391965_FacetSep_Geometry Processing and Constrained Optimization",
              "count": 18
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1164964_FacetSep_Elise Lavou\u00e9_JoinSep_413091_FacetSep_Situated Interaction, Collaboration, Adaptation and Learning",
              "count": 17
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "531974_FacetSep_Marc Plantevit_JoinSep_413086_FacetSep_Data Mining and Machine Learning",
              "count": 17
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "908599_FacetSep_Aur\u00e9lien Tabard_JoinSep_413091_FacetSep_Situated Interaction, Collaboration, Adaptation and Learning",
              "count": 17
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "167613_FacetSep_Sylvie Servigne_JoinSep_401125_FacetSep_Base de Donn\u00e9es",
              "count": 16
            },
            {
              "field": "authIdHasPrimaryStructure_fs",
              "value": "1170678_FacetSep_St\u00e9phane Derrode_JoinSep_403930_FacetSep_Extraction de Caract\u00e9ristiques et Identification",
              "count":